In [3]:
# pip install mysql-connector-python

In [4]:
import mysql.connector

# ------------------ DATABASE CONNECTION ------------------
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Siva",  # your mysql work bench password
    database="atm_db"
)
cursor = db.cursor()

# ------------------ ACCOUNT CREATION ------------------
def create_account():
    print("\n====== CREATE NEW ACCOUNT ======")

    while True:
        atm_number = input("Enter a 10-digit ATM Number: ").strip()

        if not (atm_number.isdigit() and len(atm_number) == 10):
            print("❌ ATM number must be exactly 10 digits.")
            continue

        # Check if ATM number already exists
        cursor.execute("SELECT * FROM accounts WHERE atm_number=%s", (atm_number,))
        if cursor.fetchone():
            print("❌ ATM Number already exists! Try another.")
        else:
            break

    pin = input("Create a PIN: ").strip()

    # Initial deposit
    while True:
        try:
            initial_amount = float(input("Enter amount to deposit initially: "))
            if initial_amount < 0:
                print("❌ Amount cannot be negative.")
            else:
                break
        except:
            print("❌ Enter a valid number")

    # Insert into database
    cursor.execute(
        "INSERT INTO accounts (atm_number, pin, balance) VALUES (%s, %s, %s)",
        (atm_number, pin, initial_amount)
    )
    db.commit()

    print("\n✅ Account Created Successfully!")
    print(f"💳 Your ATM Number: {atm_number}")
    print("👉 Please login to continue.\n")


# ------------------ LOGIN FUNCTION ------------------
def login():
    print("\n====== LOGIN ======")

    atm_number = input("Enter your 10-digit ATM number: ").strip()

    if not (atm_number.isdigit() and len(atm_number) == 10):
        print("❌ Invalid ATM number format.")
        return None

    pin = input("Enter your ATM PIN: ").strip()

    cursor.execute(
        "SELECT id, atm_number, pin, balance FROM accounts WHERE atm_number=%s AND pin=%s",
        (atm_number, pin)
    )
    user = cursor.fetchone()

    if user:
        print("\n✅ Login Successful!")
        return user
    else:
        print("\n❌ Incorrect ATM Number or PIN!")
        return None


# ------------------ REFRESH USER RECORD ------------------
def refresh_user(user_id):
    cursor.execute(
        "SELECT id, atm_number, pin, balance FROM accounts WHERE id=%s", (user_id,)
    )
    return cursor.fetchone()


# ------------------ WITHDRAW FUNCTION ------------------
def withdraw(user):
    try:
        amount = float(input("Enter amount to withdraw: "))
    except:
        print("❌ Enter a valid amount.")
        return user

    current_balance = float(user[3])

    if amount <= 0:
        print("❌ Amount must be greater than 0.")
        return user

    if amount > current_balance:
        print("❌ Insufficient balance!")
        return user

    new_balance = current_balance - amount

    cursor.execute(
        "UPDATE accounts SET balance=%s WHERE id=%s", (new_balance, user[0])
    )
    db.commit()

    print("\n✅ Withdrawal Successful!")
    print(f"💰 Withdrawn: {amount}")
    print(f"💳 Remaining Balance: {new_balance}")

    return refresh_user(user[0])


# ------------------ DEPOSIT FUNCTION ------------------
def deposit(user):
    try:
        amount = float(input("Enter amount to deposit: "))
    except:
        print("❌ Enter a valid amount.")
        return user

    if amount <= 0:
        print("❌ Amount must be greater than 0.")
        return user

    new_balance = float(user[3]) + amount

    cursor.execute(
        "UPDATE accounts SET balance=%s WHERE id=%s", (new_balance, user[0])
    )
    db.commit()

    print("\n✅ Deposit Successful!")
    print(f"➕ Deposited: {amount}")
    print(f"💳 Current Balance: {new_balance}")

    return refresh_user(user[0])


# ------------------ CHECK BALANCE ------------------
def check_balance(user):
    print(f"\n💳 Your Current Balance: {user[3]}")


# ------------------ MAIN ATM PROGRAM ------------------
def main():
    print("====== WELCOME TO ATM MACHINE ======")

    # Ask user to create an account
    print("\nDo you want to create a new account?")
    create_choice = input("Enter yes/no: ").lower().strip()

    if create_choice in ("yes", "y"):
        create_account()

    # Now login
    user = None
    while not user:
        user = login()

    # Run ATM operations
    while True:
        print("\nChoose an option:")
        print("1. Withdraw")
        print("2. Deposit")
        print("3. Check Balance")
        print("4. Exit")

        choice = input("\nEnter your choice: ").strip()

        if choice == "1":
            user = withdraw(user)

        elif choice == "2":
            user = deposit(user)

        elif choice == "3":
            check_balance(user)

        elif choice == "4":
            ask = input("\nDo you want to know remaining balance before exit? (yes/no): ").lower()
            if ask in ("yes", "y"):
                user = refresh_user(user[0])
                check_balance(user)

            print("\n🔒 Thank you for using the ATM! Exiting now.")
            break

        else:
            print("❌ Invalid option!")


# ------------------ RUN PROGRAM ------------------
main()


====== WELCOME TO ATM MACHINE ======

Do you want to create a new account?


Enter yes/no:  yes



====== CREATE NEW ACCOUNT ======


Enter a 10-digit ATM Number:  9573747465
Create a PIN:  2563
Enter amount to deposit initially:  5000



✅ Account Created Successfully!
💳 Your ATM Number: 9573747465
👉 Please login to continue.


====== LOGIN ======


...


✅ Withdrawal Successful!
💰 Withdrawn: 2000.0
💳 Remaining Balance: 8000.0

Choose an option:
1. Withdraw
2. Deposit
3. Check Balance
4. Exit



Enter your choice:  3



💳 Your Current Balance: 8000.00

Choose an option:
1. Withdraw
2. Deposit
3. Check Balance
4. Exit



Enter your choice:  4

Do you want to know remaining balance before exit? (yes/no):  no



🔒 Thank you for using the ATM! Exiting now.
